In [1]:
from tensorflow.keras.layers import LSTM,Input,RepeatVector,TimeDistributed,Dense,Embedding
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

2026-01-19 15:10:38.347865: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-19 15:10:38.357858: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-19 15:10:38.717578: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-19 15:10:41.841070: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
with open ("df.txt","r") as f:
    text=f.read().split(".")
print(text)

['AI redirects here', ' For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation)', '\n\nArtificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making', ' It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals', '\n\nHigh-profile applications of AI include advanced web search engines (e', 'g', ', Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e', 'g', ', Google Assistant, Siri, and Alexa); autonomous vehicles (e', 'g', ', Waymo); generative and creative tools (e', 'g', ', language models and AI art); and superhuman play and analysis in strategy games (e', 'g', ', chess 

In [3]:
token=Tokenizer()
token.fit_on_texts(text)

In [4]:
sequence=token.texts_to_sequences(text)
sequence

[[2, 60, 61],
 [21, 22, 62, 63, 2, 23, 1, 8, 7, 23],
 [8,
  7,
  2,
  24,
  4,
  64,
  3,
  65,
  25,
  5,
  66,
  67,
  68,
  69,
  26,
  27,
  7,
  28,
  6,
  13,
  29,
  70,
  71,
  30,
  1,
  72,
  73],
 [74,
  24,
  9,
  31,
  3,
  14,
  10,
  75,
  76,
  15,
  77,
  1,
  78,
  32,
  1,
  79,
  15,
  80,
  81,
  5,
  82,
  33,
  83,
  1,
  34,
  13,
  1,
  7,
  5,
  84,
  85,
  15,
  86,
  33,
  87,
  3,
  88,
  89,
  16],
 [90, 91, 17, 3, 2, 35, 36, 92, 18, 93, 11],
 [12],
 [19, 18, 94, 25, 37, 38, 95, 96, 1, 97, 98, 99, 11],
 [12],
 [19, 100, 101, 1, 102, 103, 104, 11],
 [12],
 [105, 20, 1, 106, 39, 11],
 [12],
 [40, 107, 1, 2, 108, 1, 109, 110, 1, 111, 10, 112, 113, 11],
 [12],
 [114, 1, 115],
 [116,
  117,
  2,
  17,
  41,
  42,
  118,
  6,
  2,
  9,
  119,
  3,
  120,
  121,
  2,
  43,
  122,
  123,
  44,
  17,
  124,
  125,
  45,
  126,
  2,
  127,
  128,
  129,
  130,
  131,
  46,
  1,
  132,
  46,
  133,
  42,
  134,
  2,
  135],
 [136, 137, 3, 2, 14, 41, 138, 139, 47, 16,

In [5]:
max_len = max(len(seq) for seq in sequence)
max_len

39

In [6]:
sequences=pad_sequences(sequence,padding="post",maxlen=max_len)
sequences

array([[  2,  60,  61, ...,   0,   0,   0],
       [ 21,  22,  62, ...,   0,   0,   0],
       [  8,   7,   2, ...,   0,   0,   0],
       ...,
       [ 20,  58, 233, ...,   0,   0,   0],
       [241, 242,  49, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], shape=(28, 39), dtype=int32)

In [7]:
vocab_size = len(token.word_index) + 1
vocab_size

259

In [8]:
x = sequences
x

array([[  2,  60,  61, ...,   0,   0,   0],
       [ 21,  22,  62, ...,   0,   0,   0],
       [  8,   7,   2, ...,   0,   0,   0],
       ...,
       [ 20,  58, 233, ...,   0,   0,   0],
       [241, 242,  49, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], shape=(28, 39), dtype=int32)

In [9]:
encoder = Sequential([
    Input(shape=(max_len,)),
    Embedding(input_dim=vocab_size, output_dim=64, mask_zero=True),
    LSTM(100, return_sequences=True),
    LSTM(30)
])

E0000 00:00:1768831844.831491  160404 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1768831844.837933  160404 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
decoder = Sequential([
    Input(shape=(30,)),
    RepeatVector(max_len),
    LSTM(100, return_sequences=True),
    TimeDistributed(Dense(vocab_size, activation="softmax"))
])


In [11]:
autoencoder=Sequential([
    encoder,
    decoder
])

In [12]:
autoencoder.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy"
)

In [13]:
autoencoder.fit(x,x,epochs=1000,batch_size=16)

Epoch 1/1000


2026-01-19 15:10:51.806742: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - loss: 5.5541
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 5.5357
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 5.5020
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 5.4350 
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 5.2510 
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 4.9186
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 4.3530
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 3.8375
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 3.4514
Epoch 10/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 3.1501
Epoch 11/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 3.0327
Epoch 12/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 2.9279
Epoch 13/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 2.8835
Epoch 14/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 2.8498
Epoch 15/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.8131 
Epoch 16/1000

In [14]:
pred = autoencoder.predict(x)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step


array([[[3.6765088e-03, 1.5988823e-02, 7.4776083e-02, ...,
         3.2571390e-06, 4.4026824e-06, 6.8998263e-05],
        [2.2475393e-02, 2.4084896e-01, 5.7907254e-02, ...,
         1.4649019e-07, 2.1086399e-07, 1.6638236e-05],
        [7.5815886e-02, 6.0046203e-02, 3.1108959e-03, ...,
         1.7234594e-08, 5.3526168e-08, 1.0185273e-05],
        ...,
        [9.9964190e-01, 2.2597413e-05, 2.7120330e-05, ...,
         9.3351138e-10, 1.9326742e-09, 4.4849735e-06],
        [9.9964190e-01, 2.2597413e-05, 2.7120330e-05, ...,
         9.3351138e-10, 1.9326742e-09, 4.4849735e-06],
        [9.9964190e-01, 2.2597413e-05, 2.7120330e-05, ...,
         9.3351138e-10, 1.9326742e-09, 4.4849735e-06]],

       [[3.9939879e-04, 1.7202325e-02, 6.1587565e-02, ...,
         4.9242140e-06, 2.6192015e-06, 1.5418396e-05],
        [7.0337068e-05, 4.1831914e-02, 3.6315992e-02, ...,
         5.3933185e-08, 1.4984275e-08, 1.7412732e-07],
        [6.0622275e-05, 7.2393872e-02, 6.2688969e-02, ...,
         2.490

In [15]:
pred_ids = pred.argmax(axis=-1)
pred_ids

array([[114,  60, 115, ...,   0,   0,   0],
       [ 19, 220,  62, ...,   0,   0,   0],
       [  8,   7,  24, ...,   0,   0,   0],
       ...,
       [136, 137, 233, ...,   0,   0,   0],
       [  8, 173,  24, ...,   0,   0,   0],
       [  8,   0,   0, ...,   0,   0,   0]], shape=(28, 39))

In [16]:
index_word = {v: k for k, v in token.word_index.items()}
index_word[0] = "<PAD>"
index_word

{1: 'and',
 2: 'ai',
 3: 'of',
 4: 'the',
 5: 'to',
 6: 'as',
 7: 'intelligence',
 8: 'artificial',
 9: 'a',
 10: 'in',
 11: 'e',
 12: 'g',
 13: 'learning',
 14: 'research',
 15: 'that',
 16: 'goals',
 17: 'applications',
 18: 'search',
 19: 'google',
 20: 'generative',
 21: 'for',
 22: 'other',
 23: 'disambiguation',
 24: 'is',
 25: 'systems',
 26: 'with',
 27: 'human',
 28: 'such',
 29: 'reasoning',
 30: 'perception',
 31: 'field',
 32: 'methods',
 33: 'their',
 34: 'use',
 35: 'include',
 36: 'advanced',
 37: 'used',
 38: 'by',
 39: 'tools',
 40: 'language',
 41: 'are',
 42: 'not',
 43: 'has',
 44: 'general',
 45: 'being',
 46: 'enough',
 47: 'particular',
 48: 'processing',
 49: 'have',
 50: 'techniques',
 51: 'neural',
 52: 'networks',
 53: 'create',
 54: 'an',
 55: 'funding',
 56: 'known',
 57: 'after',
 58: "ai's",
 59: 'about',
 60: 'redirects',
 61: 'here',
 62: 'uses',
 63: 'see',
 64: 'capability',
 65: 'computational',
 66: 'perform',
 67: 'tasks',
 68: 'typically',
 69: 'a

In [17]:
def decode_sequence(seq):
    words = []
    for idx in seq:
        if idx == 0:
            continue
        words.append(index_word.get(idx, "?"))
    return " ".join(words)


In [18]:
compression = encoder.predict(x)
compression.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step


(28, 30)

In [19]:
n =1

real_sentence = decode_sequence(x[n])
reconstructed_sentence = decode_sequence(pred_ids[n])

print("REAL:        ", real_sentence)
print("---------------------------------------------------------------------------------------------------------------------------------")
print("RECONSTRUCT: ", reconstructed_sentence)


REAL:         for other uses see ai disambiguation and artificial intelligence disambiguation
---------------------------------------------------------------------------------------------------------------------------------
RECONSTRUCT:  google growth uses see after linguistics philosophy and and other


### combine recurrent-autoencoder with pca for dimention reducing and visualisation

In [20]:
pca=PCA()
compression_2D=pca.fit_transform(compression)
df = pd.DataFrame({
    "x": compression_2D[:,0],
    "y": compression_2D[:,1],
    "sentence": text
})

fig = px.scatter(df, x="x", y="y", hover_data=["sentence"])
fig.show()

plt.show()